# 캘리포니아 주택 가격 예측

### 기본 정보

- 인구, 중간소득 등의 특성을 사용하여 주택 가격 예측
- 다중 회귀, 단방향 회귀(반대 : 다변량 회귀)
    - 다중 회귀 : 여러 개의 특성을 이용한 회귀
    - 단방향 회귀 : 하나의 데이터에 대한 예측
    - 다변량 회귀 : 여러 개의 수치를 예측하기 위한 회귀
- 회귀의 성능 측정 방법
    - 평균 제곱근의 오차를 활용
    - 평균 절대오차를 활용